# Proyecto - Asistente de IA para aerolíneas

Ahora, reuniremos lo que hemos aprendido para crear un asistente de IA para atención al cliente para una aerolínea.

In [ ]:
# imports

import os
import json
from dotenv import load_dotenv
from anthropic import Anthropic
import gradio as gr
import pandas as pd
import uuid  ## para identificador único
from datetime import date # para formato fecha

In [ ]:
# Inicialización

load_dotenv()

anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:8]}")
else:
    print("Anthropic API Key Sin Configurar")
    
MODEL = "claude-3-5-sonnet-20241022"
claude = Anthropic(api_key=anthropic_api_key)

In [ ]:
system_message = "Eres un asistente útil para una aerolínea llamada FlightAI. "
system_message += "Da respuestas breves y corteses, de no más de una oración. "
system_message += "Se siempre preciso. Si no sabes la respuesta, dilo."

In [ ]:
# Esta función parece bastante más simple que la de mi video, porque estamos aprovechando las últimas actualizaciones de Gradio.

def chat(message, history):
    messages = [{"role": "user" if i % 2 ==0 else 'assistant', "content": msg['content']} for i, msg in enumerate(history)]
    print(history)
    messages.append({"role": "user", "content": message})
    response = claude.messages.create(model=MODEL,
                                      system=system_message, 
                                      messages=messages,
                                      max_tokens=500,
                                      temperature=0.5,
                                      )
    return response.content[0].text

gr.ChatInterface(fn=chat, type="messages").launch()

## Herramientas

Las herramientas son una característica increíblemente poderosa que ofrecen los LLM de vanguardia.

Con las herramientas, puedes escribir una función y hacer que el LLM llame a esa función como parte de su respuesta.

Suena casi espeluznante... ¿le estamos dando el poder de ejecutar código en nuestra máquina?

Bueno, más o menos.

In [ ]:
data = {
    'Nombre': ['Juan Pérez', 'María López', 'Pedro Gómez', 'Ana Martínez'],
    'Apellido': ['Pérez', 'López', 'Gómez', 'Martínez'],
    'Destino': ['Cancún', 'Nueva York', 'París', 'Tokio'],
    'Fecha_Ida': ['2024-12-25', '2025-02-14', '2024-09-01', '2025-05-15'],
    'Fecha_Vuelta': ['2025-01-05', '2025-02-22', '2024-09-10', '2025-05-25'],
    'Número_Pasajeros': [2, 1, 3, 4],
    'Clase': ['Turista', 'Business', 'Primera', 'Turista'],
    'Precio_Total': [1000, 2500, 3000, 1500],
    'id': [uuid.uuid4() for _ in range(4)]
}
data = pd.DataFrame(data)

In [ ]:
data.head()

In [ ]:
# Vamos a hacer una función útil

ticket_prices = {"londres": "799€", "paris": "899€", "tokyo": "1400€", "berlin": "499€"}

def get_ticket_price(destination_city):
    print(f"Se solicitó la herramienta get_ticket_price para {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [ ]:
def reserva(nombre, apellido, destino, fecha_ida, fecha_vuelta, numero_pasajeros, clase, precio_total):
        global data
        print(f"Se solicitó la herramienta reserva para {nombre} {apellido}")
        temp ={'Nombre': [nombre], 'Apellido': apellido, 'Destino': destino,
        'Fecha_Ida': fecha_ida, 'Fecha_Vuelta': fecha_vuelta,
        'Número_Pasajeros': numero_pasajeros, 'Clase': clase.capitalize(), 'Precio_Total':precio_total, "id": uuid.uuid4()}
        temp =pd.DataFrame(data=temp)
        data = pd.concat([data, temp], ignore_index=True)
        return f"Se ajendo la reserva correctamente {nombre}"

In [ ]:
# Hay una estructura de diccionario particular que se requiere para describir nuestra función:

price_function = {
    "name": "get_ticket_price",
    "description": "Obtén el precio de un billete de ida y vuelta a la ciudad de destino. Llámalo siempre que necesites saber el precio del billete, por ejemplo, cuando un cliente pregunte '¿Cuánto cuesta un billete a esta ciudad?'",
    "input_schema": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "La ciudad a la que el cliente desea viajar",
            },
        },
        "required": ["destination_city"]
    }
}

In [ ]:
reserva_function = {
    "name": "reserva",
    "description": "Recolectar los datos para ajendar la reserva de avion. Llámalo siempre que necesites ajendar una reserva, cuando un cliente pregunte '¿puedo ajendar una reserva? o 'quiero ajendar un vuelo'",
    "input_schema": {
        "type": "object",
        "properties": {
            "nombre": {
                "type": "string",
                "description": "El nombre de la persona que realiza la reserva",
            },
            "apellido": {
                "type": "string",
                "description": "El apellido de la persona que realiza la reserva",
            },
            "destino": {
                "type": "string",
                "description": "La ciudad a la que el cliente desea viajar",
            },
            "fecha_ida": {
                "type": "string",
                "description": "La fecha de ida del vuelo en formato YYYY-MM-DD",
            },
            "fecha_vuelta": {
                "type": "string",
                "description": "La fecha de vuelta del vuelo en formato YYYY-MM-DD",
            },
            "numero_pasajeros": {
                "type": "integer",
                "description": "El número de pasajeros que viajarán",
            },
            "clase": {
                "type": "string",
                "description": "La clase en la que viajarán los pasajeros",
            },
            "precio_total": {
                "type": "number",
                "description": "El precio total de la reserva",
            },
        },
        "required": ["monbre", "apellido", "destino", "fecha_ida", "fecha_vuelta", "numero_pasajeros", "clase", "precio_total"]
    }
}

In [ ]:
# Y esto está incluido en una lista de herramientas:

tools = [reserva_function, price_function]

## Cómo hacer que OpenAI use nuestra herramienta

Hay algunos detalles complicados para permitir que OpenAI "llame a nuestra herramienta"

Lo que realmente hacemos es darle al LLM la oportunidad de informarnos que quiere que ejecutemos la herramienta.

Así es como se ve la nueva función de chat:

In [ ]:
def chat(message, history):
    messages = [{"role": "user" if i % 2==0 else "assistant",'content': msg['content']} for i,msg in enumerate(history)]
    messages.append({"role":"user","content":message})
    response = claude.messages.create(model=MODEL,
                                     messages=messages,
                                    system=system_message,
                                    tools=tools,
                                    max_tokens=250,
                                   )
                                       
    if response.content[0].type == 'tool_use':
        message = response.content[0]
        asistente = [{"type": message.type,
                    "id": message.id,
                    "name": message.name,
                    "input": message.input}]
        print(f"Necesidad de herramienta \n {response}")
        messages.append({"role":"assistant","content":asistente})
        response= handle_tool_call(message)
        messages.append(response)
        print(f"\n El historial  : \n {messages}")
        

        response = claude.messages.create(model=MODEL,
                                     messages=messages,
                                    system=system_message,
                                    tools=tools,
                                    max_tokens=250,
                                    temperature=0)
        return response.content[0].text
    else:
        print(f'no es tool_user {response}')
    return response.content[0].text

In [ ]:
#Tenemos que escribir esa función handle_tool_call:

def handle_tool_call(message):
    if message.name == "get_ticket_price":
        city = message.input.get('destination_city')
        price = get_ticket_price(city)
        response = {
            "role": "user",
            "content":[{'type':"tool_result","tool_use_id":message.id,"content": price}],
        }
        return response
    elif message.name == "reserva":
        nombre = message.input.get('nombre')
        apellido = message.input.get('apellido')
        destino = message.input.get('destino')
        fecha_ida = message.input.get('fecha_ida')
        fecha_vuelta = message.input.get('fecha_vuelta')
        numero_pasajeros = message.input.get('numero_pasajeros')
        clase = message.input.get('clase')
        precio_total = message.input.get('precio_total')
        reserva(nombre, apellido, destino, fecha_ida, fecha_vuelta, numero_pasajeros, clase, precio_total)
        response = {
            "role": "user",
            "content": [{'type': "tool_result", "tool_use_id": message.id, "content": f"Se agendó la reserva correctamente para {nombre}"}],
        }
        return response


In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
# veamos los dato de la reserva
data